Ax =1-b  <--- find a solution with smallest hamming weight? 

In [ ]:


def weak_compositions(N: int, k: int):
    """
    Yield all weak compositions of N into exactly k parts.
    Parts are >= 0.
    """
    # choose k-1 bar positions among N+k-1 slots
    for bars in combinations(range(N + k - 1), k - 1):
        prev = -1
        parts = []
        for b in bars:
            parts.append(b - prev - 1)
            prev = b
        parts.append(N + k - 1 - prev - 1)
        yield tuple(parts)

def bounded_compositions(total, caps):
    """
    Yield tuples x of len(caps) such that:
      0 <= x[k] <= caps[k] and sum(x) == total
    Prunes early.
    """
    caps = tuple(caps)
    n = len(caps)

    def rec(i, remaining, prefix):
        if i == n - 1:
            if 0 <= remaining <= caps[i]:
                yield prefix + (remaining,)
            return
        # x_i can be at most min(cap, remaining)
        hi = min(caps[i], remaining)
        for x in range(hi + 1):
            # quick feasibility check: can the rest still sum to remaining-x?
            # lower bound is 0, upper bound is sum(caps[i+1:])
            rem2 = remaining - x
            if rem2 < 0:
                break
            if rem2 > sum(caps[i+1:]):
                continue
            yield from rec(i + 1, rem2, prefix + (x,))

    yield from rec(0, total, ())

In [225]:
appx_max_rank = 0

for _, buttons, req in cases:
    appx_rank = len(buttons)-len(req)
    if appx_rank > appx_max_rank:
        appx_max_rank = appx_rank
        print(f"approximate max_rank for buttons {appx_max_rank}")

    # So the input max rank is about 3. 
    # That's easy, the problem is solved on a very small subspace.

approximate max_rank for buttons 2
approximate max_rank for buttons 3


In [226]:
from fractions import Fraction
import sympy as sp

def to_Ab_linear(ineqs, taus):
    """
    Convert a list of linear sympy inequalities to A*tau <= b with rational Fractions.
    Supports only <= or >= and linear expressions.
    """
    A = []
    b = []
    taus = list(taus)

    for ineq in ineqs:
        expr = sp.expand(ineq.lhs - ineq.rhs)  # expr rel 0
        rel = ineq.rel_op

        # normalize to <= 0
        if rel == ">=":
            expr = -expr
        elif rel == "<=":
            pass
        else:
            raise ValueError(f"Unsupported rel {rel}")

        coeffs = [sp.Rational(expr.coeff(t)) for t in taus]
        const = sp.Rational(expr.subs({t: 0 for t in taus}))

        # expr = sum coeffs*tau + const <= 0  =>  sum coeffs*tau <= -const
        A.append([Fraction(int(c.p), int(c.q)) for c in coeffs])
        rhs = -const
        b.append(Fraction(int(rhs.p), int(rhs.q)))

    return A, b

def fm_eliminate_one(A, b, elim_idx):
    """
    Eliminate variable elim_idx from A*x <= b (Fourier–Motzkin).
    """
    P, N, Z = [], [], []
    for row, rhs in zip(A, b):
        c = row[elim_idx]
        if c > 0:
            P.append((row, rhs))
        elif c < 0:
            N.append((row, rhs))
        else:
            Z.append((row, rhs))

    A2, b2 = [], []
    # keep Z constraints (drop column)
    for row, rhs in Z:
        A2.append(row[:elim_idx] + row[elim_idx+1:])
        b2.append(rhs)

    # combine each P with each N
    for rowp, rhsp in P:
        cp = rowp[elim_idx]  # >0
        for rown, rhsn in N:
            cn = rown[elim_idx]  # <0
            newrow = []
            for j in range(len(rowp)):
                if j == elim_idx:
                    continue
                newrow.append(rowp[j]/cp - rown[j]/cn)
            newrhs = rhsp/cp - rhsn/cn
            A2.append(newrow)
            b2.append(newrhs)

    return A2, b2

def fm_bounds_for_var(A, b, var_idx):
    """
    Eliminate all variables except var_idx, returning lower/upper bounds on that variable.
    Returns (lo, hi) as Fractions, possibly infinite (None).
    """
    n = len(A[0])
    # Bring var_idx to last position for easier elimination
    perm = [j for j in range(n) if j != var_idx] + [var_idx]

    def permute_row(row):
        return [row[j] for j in perm]

    Acur = [permute_row(row) for row in A]
    bcur = list(b)

    # eliminate variables 0..n-2 (keep last)
    for elim in range(n-1):
        Acur, bcur = fm_eliminate_one(Acur, bcur, 0)  # always eliminate first var

    # Now system is 1D: a * t <= rhs
    lo = None  # lower bound
    hi = None  # upper bound
    for row, rhs in zip(Acur, bcur):
        a = row[0]
        if a == 0:
            if rhs < 0:
                return None, None  # infeasible
            continue
        bound = rhs / a
        if a > 0:
            # t <= bound
            hi = bound if hi is None else min(hi, bound)
        else:
            # t >= bound  (since dividing flips)
            lo = bound if lo is None else max(lo, bound)
    return lo, hi



In [ ]:
import re
from itertools import combinations
from sympy.solvers.inequalities import reduce_inequalities
import numpy as np

for _, buttons, req in cases:
    R = len(req)
    B = len(buttons)

    M = sp.zeros(R, B)
    for j, indices in enumerate(buttons):
        for i in indices:
            M[i, j] = 1

    b = sp.Matrix(req)
    solution, params = M.gauss_jordan_solve(b)
    
    if len(params) == 0:
        print("Unique solution")
        print(solution)

    print(f"Number of free parameters: {len(params)}")
    if len(params)==0:
        print("No free parameters, skipping")
        continue
    print(solution)
    print(req)

    # How do I find cone where all solution variables are nonnegative and bounded above by req?    
    upper_bounds = [min(req[i] for i in buttons[j]) for j in range(len(buttons))] # upper bound on button presses
    
    # Setup inequalities and find all inputs that satisfy them
    lower_bounds_ineq = [0 <= solution[i] for i in range(len(solution))]
    upper_bounds_ineq = [solution[i] <= upper_bounds[i] for i in range(len(solution))]
    all_ineq = lower_bounds_ineq + upper_bounds_ineq
    all_ineq = [ineq for ineq in all_ineq if ineq.free_symbols]  # drop trivial ones

    # Collect all symbols
    all_symbols = set()
    for ineq in all_ineq:
        all_symbols.update(ineq.free_symbols)

    # Solve inequalities
    A,b = to_Ab_linear(all_ineq, list(all_symbols))

    taus = list(params)  # sympy symbols
    A, b = to_Ab_linear(all_ineq, taus)

    # Iterate over 

    






    
    
       
        






Unique solution
Matrix([[18], [1], [0], [9], [7], [16], [8], [12], [17]])
Number of free parameters: 0
No free parameters, skipping
Number of free parameters: 2
Matrix([[5], [tau1 + 5], [2*tau0 - tau1 - 16], [71 - 3*tau0], [65 - 3*tau0], [tau0 - 1], [tau0 - 18], [tau0], [tau1]])
[65, 52, 55, 5, 53, 42, 49]
tau0 18 65/3
tau1 0 82/3
Unique solution
Matrix([[13], [18], [18], [4], [11]])
Number of free parameters: 0
No free parameters, skipping
Unique solution
Matrix([[13], [8], [0], [15], [3], [14]])
Number of free parameters: 0
No free parameters, skipping
Number of free parameters: 1
Matrix([[39/2 - tau0/2], [16], [35/2 - tau0/2], [27/2 - tau0/2], [tau0/2 + 3/2], [49/2 - tau0/2], [tau0/2 + 25/2], [9], [tau0]])
[86, 57, 60, 72, 9, 70, 49, 74]
tau0 0 27
Number of free parameters: 2
Matrix([[4*tau0/19 + 11*tau1/19 + 37/19], [-9*tau0/19 + 18*tau1/19 - 88/19], [-4*tau0/19 - 11*tau1/19 + 533/19], [-3*tau0/19 + 6*tau1/19 + 148/19], [11*tau0/19 - 3*tau1/19 + 154/19], [-8*tau0/19 - 3*tau1/19 + 4

In [ ]:
import re
from itertools import combinations
from ortools.sat.python import cp_model

def parse_buttons(buttons): 
    # Get button vectors 
    button_indices = []
    for b in buttons:
            indices = [int(x) for x in b.strip('()').split(',')]
            button_indices.append(tuple(indices))

    return button_indices

cases = []
with open('input.txt', 'r') as f:
    data = f.readlines()
    for k in range(len(data)):
        diagram = re.findall('\[.*\]', data[k])[0].strip('[]')
        buttons = re.findall('\(\d[,\d]*\)', data[k])
        reqs = [int(x) for x in re.findall('\{.*\}', data[k])[0].strip('{}').split(',')]

        buttons = parse_buttons(buttons)
        cases.append((diagram, buttons, reqs))

_, buttons, req = cases[0]

def solve_buttons_cpsat(req, buttons):
    """
    req: list[int] length R
    buttons: list[tuple[int]] length B; button j affects requirements in buttons[j]
    """
    R = len(req)
    B = len(buttons)

    # Upper bounds per button (safe)
    ub = [min(req[i] for i in idxs) if idxs else 0 for idxs in buttons]

    model = cp_model.CpModel()
    x = [model.NewIntVar(0, ub[j], f"x{j}") for j in range(B)]

    # Constraints: sum_{j affecting i} x_j == req[i]
    for i in range(R):
        model.Add(sum(x[j] for j in range(B) if i in buttons[j]) == req[i])

    # Objective: minimize total presses
    model.Minimize(sum(x))

    solver = cp_model.CpSolver()
    solver.parameters.max_time_in_seconds = 10.0  # optional
    status = solver.Solve(model)

    if status in (cp_model.OPTIMAL, cp_model.FEASIBLE):
        sol = [solver.Value(xj) for xj in x]
        return sol, sum(sol), status
    return None, None, status

total_presses = 0
for _, buttons, req in cases:
    sol, total, status = solve_buttons_cpsat(req, buttons)
    if sol is not None:
        total_presses += total
    else:
        print(f"No solution found, status {status}")

total_presses

21021

In [239]:
pip install ortools

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 20.2/20.2 MB 8.9 MB/s eta 0:00:0000:0100:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 10.8/10.8 MB 10.9 MB/s eta 0:00:0000:0100:01
  Using cached absl_py-2.3.1-py3-none-any.whl (135 kB)
  Using cached typing_extensions-4.15.0-py3-none-any.whl (44 kB)
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 425.6/425.6 KB 8.8 MB/s eta 0:00:00a 0:00:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 348.5/348.5 KB 8.5 MB/s eta 0:00:0000:01
  Using cached pytz-2025.2-py2.py3-none-any.whl (509 kB)
You should consider upgrading via the '/usr/local/bin/python3 -m pip install --upgrade pip' command.
Note: you may need to restart the kernel to use updated packages.
